## 2025 1. 26

根据bennet fast relax + hhj 脚本进行threading - rosetta repack - fast relax的尝试

In [1]:
from pyrosetta import *
from pyrosetta.rosetta import *

In [ ]:
init( "-beta_nov16 -in:file:silent_struct_type binary -mute all" +
    " -use_terminal_residues true -mute basic.io.database core.scoring" )


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.conda.ubuntu.cxx11thread.serialization.Ubuntu.python38.Release 2025.03+release.1f5080a079a5261122c0e532c46f61a4f7e20df8 2025-01-13T15:37:50] retrieved from: http://www.pyrosetta.org


In [7]:
# Loading xml and mover
xml = os.path.join('RosettaFastRelaxUtil.xml')
objs = protocols.rosetta_scripts.XmlObjects.create_from_file(xml)

detect_symm = objs.get_mover('detect_symm')
packer = objs.get_mover('repack')
FastRelax = objs.get_mover('FastRelax')

In [8]:
# loading pose
pose = pose_from_pdb('fast_relax_folder/TMHC8.pdb')
print('Detecting symmetry...')
detect_symm.apply(pose)

Detecting symmetry...


In [12]:
alpha_1 = list("ARNDCQEGHILKMFPSTWYV-")
states = len(alpha_1)
alpha_3 = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE',
           'LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL','GAP']

aa_1_N = {a:n for n,a in enumerate(alpha_1)}
aa_3_N = {a:n for n,a in enumerate(alpha_3)}
aa_N_1 = {n:a for n,a in enumerate(alpha_1)}
aa_1_3 = {a:b for a,b in zip(alpha_1,alpha_3)}
aa_3_1 = {b:a for a,b in zip(alpha_1,alpha_3)}


In [13]:
rsd_set = pose.residue_type_set_for_pose(core.chemical.FULL_ATOM_t )
nemo_seq = "GARRRLRLLFLILLLVFLLFILLAFLLLLLIFVFFYKMREEIKKLEKEGAASGREIAKMEMDHYNNLLDHVLRLSAIFSDSIIRLLELLMRLVEMILEYA" * 8 
for resi, mut_to in enumerate(nemo_seq):
    resi += 1 # 1 indexing
    name3 = aa_1_3[ mut_to ]
    new_res = core.conformation.ResidueFactory.create_residue( rsd_set.name_map( name3 ) )
    pose.replace_residue( resi, new_res, True )
# save the pose
pose.dump_pdb('fast_relax_folder/TMHC8_mutated.pdb')

True

In [ ]:
# Repacking
print("threading mpnn sequence...")
packer.apply(pose)
# save the pose
pose.dump_pdb('fast_relax_folder/TMHC8_mutated_repacked.pdb')

# FastRelax
print('Running FastRelax...')
FastRelax.apply(pose)
# save the pose
pose.dump_pdb('fast_relax_folder/TMHC8_mutated_repacked_fastrelaxed.pdb')

True